In [30]:
import numpy as np

def objective_function(t, N, K, r, t0):
    part1 = r * t * (N - (N / K) * t) + 1 / (1 + np.exp(-r * (t - t0)))
    return part1


In [31]:
def normalize(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)

In [32]:
from random import random
from itertools import accumulate
from bisect import bisect_right
from gaft.plugin_interfaces.operators.selection import Selection

class LinearRankingSelection(Selection):
    def __init__(self, pmin=0.1, pmax=0.9):
        '''
        LinearRankingSelection 类，使用线性排名选择方法的选择算子。
        参考文献: Baker J E. 自适应遗传算法选择方法[C]//遗传算法及其应用国际会议论文集. 1985: 101-111。
        '''
        # 最差和最优个体的选择概率。
        self.pmin, self.pmax = pmin, pmax

    def select(self, population, fitness):
        '''
        使用线性排名方法选择一对父代个体。
        '''
        # 个体数量。
        NP = len(population)
        # 给所有个体按适应度降序排列，并为其分配排名。
        sorted_indvs = sorted(population.individuals, key=fitness, reverse=True)
        # 线性分配选择概率。
        # 注意: 这里排名 i 属于 {1, ..., N}
        p = lambda i: (self.pmin + (self.pmax - self.pmin)*(i-1)/(NP-1))
        probabilities = [self.pmin] + [p(i) for i in range(2, NP)] + [self.pmax]
        # 归一化概率。
        psum = sum(probabilities)
        wheel = list(accumulate([p/psum for p in probabilities]))
        # 选择父代。
        father_idx = bisect_right(wheel, random())
        father = population[father_idx]
        mother_idx = (father_idx + 1) % len(wheel)
        mother = population[mother_idx]
        return father, mother


In [33]:
from gaft import GAEngine
from gaft.components import DecimalIndividual
from gaft import Population
from gaft.operators import RouletteWheelSelection, UniformCrossover, FlipBitMutation

# Create a DecimalIndividual template
indv_template = DecimalIndividual(ranges=[(lower_bound, upper_bound)], eps=1e-2)

# Create a Population object with the individual template
population = Population(indv_template=indv_template)

# Create genetic algorithm engine with required components
ga = GAEngine(
    population=population,
    selection=RouletteWheelSelection,
    crossover=UniformCrossover,
    mutation=FlipBitMutation
)

# Set up the parameters
ga.population.size = 50

# Set up the fitness function using the normalized objective function
@ga.fitness_register
def fitness(indv):
    t = indv.solution[0]
    normalized_fitness = normalize(objective_function(t, N, K, r, t0), min_val, max_val)
    return normalized_fitness

# Run the genetic algorithm
ga.run(generation=100)
best_individual = ga.output.get()

# Access the best individual's solution
best_solution = best_individual.solution


ImportError: cannot import name 'Population' from 'gaft' (/home/cr9294/.virtualenvs/PIXIU/lib/python3.10/site-packages/gaft/__init__.py)

In [ ]:
from random import random
from itertools import accumulate
from bisect import bisect_right
from gaft.plugin_interfaces.operators.selection import Selection

class ExponentialRankingSelection(Selection): 
    def __init__(self, base=0.5):
        '''
        ExponentialRankingSelection 类，使用指数排名选择方法的选择算子。
        :param base: 指数的底
        :type base: float 在范围 (0.0, 1.0) 内
        '''
        if not (0.0 < base < 1.0):
            raise ValueError('指数的底 base 必须在范围 (0.0, 1.0) 内')
        self.base = base

    def select(self, population, fitness):
        '''
        使用指数排名方法选择一对父代个体。
        '''
        # 个体数量。
        NP = len(population)
        # 注意: 这里排名 i 属于 {1, ..., N}
        p = lambda i: self.base**(NP - i)
        probabilities = [p(i) for i in range(1, NP + 1)]
        # 归一化概率。
        psum = sum(probabilities)
        wheel = list(accumulate([p/psum for p in probabilities]))
        # 选择父代。
        father_idx = bisect_right(wheel, random())
        father = population[father_idx]
        mother_idx = (father_idx + 1) % len(wheel)
        mother = population[mother_idx]
        return father, mother
